### Merge Shot locations and play by play

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

# import files to merge
shot_locations = pd.read_csv("../data/processed/Shot_Locations_top_20_players_2000to2020.csv")
play_by_play = pd.read_csv("../data/processed/PlaybyPlay_2000-2020.csv")

# merge
df_merged = shot_locations.merge(play_by_play, how='outer', left_on=['Game ID','Game Event ID'], right_on=['GAME_ID','EVENTNUM'])

# save as csv
df_merged.to_csv('../data/processed/PlaybyPlay_ShotLocations.csv')

In [2]:
print("Play by play shape:", play_by_play.shape)
print("Play by play games:", play_by_play['GAME_ID'].nunique())
print("Shot locations shape:", shot_locations.shape)
print("Shot locations games:", shot_locations['Game ID'].nunique())
print("Merged shaped:", df_merged.shape)

Play by play shape: (467751, 17)
Play by play games: 13980
Shot locations shape: (334331, 20)
Shot locations games: 13927
Merged shaped: (467751, 37)


In [3]:
# check if observations are missing on the play by play part
len(df_merged[df_merged['GAME_ID'].isna()])

0

In [4]:
# check if observations are missing on the shot location part
len(df_merged[df_merged['Game ID'].isna()])

132993

In [5]:
df_merged[df_merged['Game ID'].isna()].free_throw.value_counts(normalize=True)

free_throw
1.0    0.995331
0.0    0.004669
Name: proportion, dtype: float64

In [6]:
# > 99.5% of missing observations are free throws, we can try to add the shot locations manually

### Merge players stats per year

In [44]:
df_sl_pbp = pd.read_csv("../data/processed/PlaybyPlay_ShotLocations.csv", index_col=0)
df_players = pd.read_csv("../data/processed/stat_joueurs.csv", index_col=0)

In [45]:
# Fill Year feature with data from complete lines 
games = df_sl_pbp[['GAME_ID', 'Year']].drop_duplicates().dropna()

for _, game in games.iterrows():
    df_sl_pbp.loc[df_sl_pbp.GAME_ID==game['GAME_ID'], 'Year']=game['Year']

In [47]:
df_sl_pbp.Year.isna().sum()

429

In [48]:
df_sl_pbp.head(2)

,Game ID,Game Event ID,Player ID,Player Name,Team ID,Team Name,Period,Minutes Remaining,Seconds Remaining,Shot Zone Basic,Shot Zone Area,Shot Zone Range,Shot Distance,X Location,Y Location,Shot Made Flag,Home Team,Away Team,Season Type,Year,GAME_ID,EVENTNUM,target,PERIOD,PERSON1TYPE,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,at_home,3PT,jump_shot,layup_shot,dunk_shot,hook_shot,free_throw,minutes_left,seconds_left,VIDEO_AVAILABLE_FLAG
0,20000001.0,11.0,947.0,Allen Iverson,1.610613e+09,Philadelphia 76ers,1.0,11.0,8.0,4.0,2.0,0.0,19.0,-107.0,167.0,0.0,NYK,PHI,1.0,2000.0,20000001,11,0.0,1,5.0,Allen Iverson,PHI,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11,668,NaN
1,20000001.0,32.0,947.0,Allen Iverson,1.610613e+09,Philadelphia 76ers,1.0,7.0,57.0,4.0,2.0,0.0,21.0,-115.0,177.0,1.0,NYK,PHI,1.0,2000.0,20000001,32,1.0,1,5.0,Allen Iverson,PHI,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7,477,NaN


In [49]:
df_players.head(2)

,Player,Year,Age,TS%,PTM,TRBM,USG%,FG%,2P%,3P%,FT%,PTS,year_start,year_end,height,weight,C,PF,PG,PG-SG,SF,SF-SG,SG,SG-PG
38,Allen Iverson,2000.0,24.0,0.496,28.414286,3.814286,34.4,0.421,0.435,0.341,0.713,1989.0,1997,2010,182.88,165.0,0,0,0,0,0,0,1,0
48,Allen Iverson,2001.0,25.0,0.518,31.084507,3.845070,35.9,0.420,0.441,0.320,0.814,2207.0,1997,2010,182.88,165.0,0,0,0,0,0,0,1,0


In [50]:
# Check if all 20 players are there
df_players.groupby('Player')['Year'].count()

Player
Allen Iverson            11
Chris Webber              9
Dirk Nowitzki            18
Dwight Howard            16
Dwyane Wade              14
Giannis Antetokounmpo     7
James Harden             11
Jason Kidd               14
Jimmy Butler              9
Kawhi Leonard             9
Kevin Durant             12
Kevin Garnett            17
Kobe Bryant              17
LeBron James             17
Paul Pierce              18
Russell Westbrook        12
Shaquille O'Neal         12
Stephen Curry            11
Steve Nash               15
Tim Duncan               17
Name: Year, dtype: int64

In [51]:
df_sl_pbp.shape

(467751, 37)

In [52]:
# merge everything
data = df_sl_pbp.merge(df_players, how='left', left_on=['PLAYER1_NAME', 'Year'], right_on=['Player', 'Year'])

In [56]:
# drop duplicated columns
data = data.drop(['VIDEO_AVAILABLE_FLAG', 'Game ID', 'Game Event ID', 'Player Name', 'Player', 'Team Name', 'Period', 'Player ID', 'Team ID' ], axis = 1)

### Deal with NAs

In [57]:
data.isna().sum()

Minutes Remaining            132993
Seconds Remaining            132993
Shot Zone Basic              132993
Shot Zone Area               132993
Shot Zone Range              132993
Shot Distance                132993
X Location                   132993
Y Location                   132993
Shot Made Flag               132993
Home Team                    132993
Away Team                    132993
Season Type                  132993
Year                            429
GAME_ID                           0
EVENTNUM                          0
target                            0
PERIOD                            0
PERSON1TYPE                       0
PLAYER1_NAME                      0
PLAYER1_TEAM_ABBREVIATION         0
at_home                           0
3PT                               0
jump_shot                         0
layup_shot                        0
dunk_shot                         0
hook_shot                         0
free_throw                        0
minutes_left                

In [58]:
# update shot location for all free throws 
data.loc[data['free_throw']==1, 'Shot Zone Basic'] = 4.0
data.loc[data['free_throw']==1, 'Shot Zone Area'] = 1.0
data.loc[data['free_throw']==1, 'Shot Zone Range'] = 2.0
data.loc[data['free_throw']==1, 'Shot Distance'] = 15.0
data.loc[data['free_throw']==1, 'X Location'] = 0
data.loc[data['free_throw']==1, 'Y Location'] = 378

In [59]:
# drop Shot Made Flag : target is the same with no NAs
data.drop('Shot Made Flag', axis=1, inplace = True)

# drop Home Team : PLAYER1_TEAM_ABBREVIATION is the same with no NAs
data.drop('Home Team', axis=1, inplace = True)

# drop Minutes Remaining : minutes_left	is the same with no NAs
data.drop('Minutes Remaining', axis=1, inplace = True)

In [60]:
# Fill "Seconds Remaining"
data["Seconds Remaining"] = data.seconds_left - 60 * data.minutes_left

In [61]:
# Fill Season Type and Away Team with data from complete lines
games = data[['GAME_ID', 'Away Team', 'Season Type']].drop_duplicates().dropna()

for _, game in games.iterrows():
    data.loc[data.GAME_ID==game['GAME_ID'], 'Away Team']=game['Away Team']
    data.loc[data.GAME_ID==game['GAME_ID'], 'Season Type']=game['Season Type']


In [62]:
data[data.Age.isna()][['Year', 'PLAYER1_NAME']].drop_duplicates().dropna()

,Year,PLAYER1_NAME
58848,2003.0,Dwyane Wade
59135,2003.0,LeBron James
77448,2004.0,Dwight Howard
146335,2007.0,Kevin Durant
168125,2008.0,Russell Westbrook
191370,2009.0,James Harden
191409,2009.0,Stephen Curry
237119,2011.0,Kawhi Leonard
276398,2013.0,Giannis Antetokounmpo
361567,2018.0,Dwyane Wade


In [63]:
data.head(10)

,Seconds Remaining,Shot Zone Basic,Shot Zone Area,Shot Zone Range,Shot Distance,X Location,Y Location,Away Team,Season Type,Year,GAME_ID,EVENTNUM,target,PERIOD,PERSON1TYPE,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,at_home,3PT,jump_shot,layup_shot,dunk_shot,hook_shot,free_throw,minutes_left,seconds_left,Age,TS%,PTM,TRBM,USG%,FG%,2P%,3P%,FT%,PTS,year_start,year_end,height,weight,C,PF,PG,PG-SG,SF,SF-SG,SG,SG-PG
0,8,4.0,2.0,0.0,19.0,-107.0,167.0,PHI,1.0,2000.0,20000001,11,0.0,1,5.0,Allen Iverson,PHI,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11,668,24.0,0.496,28.414286,3.814286,34.4,0.421,0.435,0.341,0.713,1989.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,57,4.0,2.0,0.0,21.0,-115.0,177.0,PHI,1.0,2000.0,20000001,32,1.0,1,5.0,Allen Iverson,PHI,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7,477,24.0,0.496,28.414286,3.814286,34.4,0.421,0.435,0.341,0.713,1989.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,23,4.0,5.0,0.0,16.0,165.0,13.0,PHI,1.0,2000.0,20000001,34,0.0,1,5.0,Allen Iverson,PHI,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7,443,24.0,0.496,28.414286,3.814286,34.4,0.421,0.435,0.341,0.713,1989.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,12,4.0,1.0,2.0,15.0,0.0,378.0,PHI,1.0,2000.0,20000001,39,1.0,1,5.0,Allen Iverson,PHI,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7,432,24.0,0.496,28.414286,3.814286,34.4,0.421,0.435,0.341,0.713,1989.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,12,4.0,1.0,2.0,15.0,0.0,378.0,PHI,1.0,2000.0,20000001,40,1.0,1,5.0,Allen Iverson,PHI,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7,432,24.0,0.496,28.414286,3.814286,34.4,0.421,0.435,0.341,0.713,1989.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,57,2.0,1.0,2.0,14.0,-65.0,127.0,PHI,1.0,2000.0,20000001,51,0.0,1,5.0,Allen Iverson,PHI,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5,357,24.0,0.496,28.414286,3.814286,34.4,0.421,0.435,0.341,0.713,1989.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,16,4.0,1.0,2.0,15.0,0.0,378.0,PHI,1.0,2000.0,20000001,68,1.0,1,5.0,Allen Iverson,PHI,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4,256,24.0,0.496,28.414286,3.814286,34.4,0.421,0.435,0.341,0.713,1989.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,16,4.0,1.0,2.0,15.0,0.0,378.0,PHI,1.0,2000.0,20000001,70,1.0,1,5.0,Allen Iverson,PHI,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4,256,24.0,0.496,28.414286,3.814286,34.4,0.421,0.435,0.341,0.713,1989.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,58,6.0,5.0,1.0,23.0,235.0,40.0,PHI,1.0,2000.0,20000001,85,0.0,1,5.0,Allen Iverson,PHI,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2,178,24.0,0.496,28.414286,3.814286,34.4,0.421,0.435,0.341,0.713,1989.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,52,5.0,1.0,4.0,0.0,0.0,0.0,PHI,1.0,2000.0,20000001,87,1.0,1,5.0,Allen Iverson,PHI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,172,24.0,0.496,28.414286,3.814286,34.4,0.421,0.435,0.341,0.713,1989.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [65]:
data.to_csv('../data/processed/all_shots_2000-2020.csv')

In [64]:
data.isna().sum()

Seconds Remaining                0
Shot Zone Basic                621
Shot Zone Area                 621
Shot Zone Range                621
Shot Distance                  621
X Location                     621
Y Location                     621
Away Team                      429
Season Type                    429
Year                           429
GAME_ID                          0
EVENTNUM                         0
target                           0
PERIOD                           0
PERSON1TYPE                      0
PLAYER1_NAME                     0
PLAYER1_TEAM_ABBREVIATION        0
at_home                          0
3PT                              0
jump_shot                        0
layup_shot                       0
dunk_shot                        0
hook_shot                        0
free_throw                       0
minutes_left                     0
seconds_left                     0
Age                           6823
TS%                           6823
PTM                 

In [68]:
data[(data["3P%"].isna()) & (data["PLAYER1_NAME"]!="Shaquille O'Neal")].sort_values(by="PLAYER1_NAME")

,Seconds Remaining,Shot Zone Basic,Shot Zone Area,Shot Zone Range,Shot Distance,X Location,Y Location,Away Team,Season Type,Year,GAME_ID,EVENTNUM,target,PERIOD,PERSON1TYPE,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,at_home,3PT,jump_shot,layup_shot,dunk_shot,hook_shot,free_throw,minutes_left,seconds_left,Age,TS%,PTM,TRBM,USG%,FG%,2P%,3P%,FT%,PTS,year_start,year_end,height,weight,C,PF,PG,PG-SG,SF,SF-SG,SG,SG-PG
160573,13,4.0,1.0,2.0,15.0,0.0,378.0,GSW,1.0,2008.0,20700786,271,0.0,3,5.0,Chris Webber,GSW,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6,373,34.0,0.482,3.888889,3.555556,15.8,0.484,0.484,NaN,0.417,35.0,1994.0,2008.0,210.312,245.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
160568,28,4.0,4.0,0.0,18.0,97.0,153.0,GSW,1.0,2008.0,20700786,239,0.0,3,5.0,Chris Webber,GSW,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10,628,34.0,0.482,3.888889,3.555556,15.8,0.484,0.484,NaN,0.417,35.0,1994.0,2008.0,210.312,245.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
160567,38,4.0,5.0,0.0,20.0,202.0,-14.0,GSW,1.0,2008.0,20700786,31,1.0,1,5.0,Chris Webber,GSW,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8,518,34.0,0.482,3.888889,3.555556,15.8,0.484,0.484,NaN,0.417,35.0,1994.0,2008.0,210.312,245.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
160566,35,4.0,5.0,0.0,20.0,207.0,3.0,GSW,1.0,2008.0,20700786,8,0.0,1,5.0,Chris Webber,GSW,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10,635,34.0,0.482,3.888889,3.555556,15.8,0.484,0.484,NaN,0.417,35.0,1994.0,2008.0,210.312,245.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
162517,17,4.0,1.0,2.0,15.0,0.0,378.0,POR,1.0,2008.0,20700884,63,1.0,1,4.0,Chris Webber,GSW,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5,317,34.0,0.482,3.888889,3.555556,15.8,0.484,0.484,NaN,0.417,35.0,1994.0,2008.0,210.312,245.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196770,1,5.0,1.0,4.0,0.0,0.0,0.0,GSW,1.0,2009.0,20900298,227,1.0,2,5.0,Stephen Curry,GSW,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
386200,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21900009,209,1.0,2,0.0,Stephen Curry,DAL,1.0,1.0,1.0,0.0,0.0,0.0,0.0,9,548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
386201,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21900009,213,0.0,2,0.0,Stephen Curry,DAL,1.0,0.0,0.0,1.0,0.0,0.0,0.0,8,524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198594,18,4.0,2.0,0.0,22.0,-117.0,192.0,WAS,1.0,2009.0,20900383,20,1.0,1,4.0,Stephen Curry,GSW,1.0,0.0,1.0,0.0,0.0,0.0,0.0,10,618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
